<a href="https://colab.research.google.com/github/gloriaconcepto/ML-NLP-LEARNING/blob/main/prediciton_reddits_loneliness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [5]:
!pip install transformers

  Using cached transformers-4.35.1-py3-none-any.whl (7.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.7 MB/s eta 0:00:00


In [6]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import pandas as pd
from tqdm import tqdm

In [8]:
#prepare a small dataset that will be used for model training...
csv_file_path = '/content/drive/MyDrive/HowardNLPResearch/reddit-loneliness-project/loneliness-dataset.csv'
num_row=2500
original_df = pd.read_csv(csv_file_path)
# Extract the first 2500 rows
first_2500_rows = original_df.head(num_row)
# Extract the last 2500 rows
last_2500_rows = original_df.tail(num_row)
# Combine the subsets into a new DataFrame
new_df = pd.concat([first_2500_rows, last_2500_rows])
# Write the new DataFrame to a new CSV file
new_df.to_csv('/content/drive/MyDrive/HowardNLPResearch/reddit-loneliness-project/small-loneliness-dataset.csv', index=False)

new_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 132215
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  5000 non-null   object
 1   label    5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 117.2+ KB


In [3]:
_r# Load a large CSV file in chunks
chunk_size = 10000

data_file_path = '/content/drive/MyDrive/HowardNLPResearch/reddit-loneliness-project/loneliness-dataset.csv'

column_names = ['message', 'label']

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Initialize the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)
print(model)
# Define your optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [1]:
# Training loop
from sklearn.model_selection import train_test_split
epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


for epoch in range(epochs):
    # Read the CSV file in chunks
    for chunk in tqdm(pd.read_csv(csv_file_path, chunksize=20000, names=column_names), desc=f"Epoch {epoch+1}/{epochs}"):
        # Process each chunk
        texts = chunk['message'].tolist()
        labels = chunk['label'].tolist()

        # Tokenize the input texts using BERT tokenizer
        tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

        # Convert labels to a PyTorch tensor
        int_labels = [0]
        for i in range(1,len(labels)):
             int_labels.append(i)

        labels = torch.tensor(int_labels)
        print(labels)
        # Combine tokenized texts and labels into a TensorDataset
        dataset = TensorDataset(tokenized_texts['input_ids'], tokenized_texts['attention_mask'], labels)

        # Split the dataset into training and validation sets
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        # train_dataset, val_dataset = train_test_split(texts, labels, test_size=0.2, random_state=42)

        # Create DataLoader instances for training and validation
        train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

        # Training loop for the chunk
        model.train()
        for batch in tqdm(train_dataloader, desc="Training"):
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels.float())
            loss = outputs.loss
            loss.backward()
            optimizer.step()

#            criterion = torch.nn.CrossEntropyLoss()
# labels = labels.float()  # Convert labels to float if needed
# loss = criterion(outputs, labels)

        # Validation for the chunk
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc="Validation"):
                input_ids, attention_mask, labels = batch
                input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels.float())
                total_val_loss += outputs.loss.item()

        average_val_loss = total_val_loss / len(val_dataloader)
        print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {average_val_loss}")

# Your model is now trained!
torch.save(model.state_dict(), '/content/drive/MyDrive/HowardNLPResearch/reddit-loneliness-project/model.pth')

model.save_pretrained('/content/drive/MyDrive/HowardNLPResearch/reddit-loneliness-project/model2.pth')

NameError: ignored